# 🔋 배터리 데이터 전처리 - 새로운 구조

개선된 프로젝트 구조에서 배터리 데이터를 처리하고 채널별로 분리 관리하는 예제입니다.

## 📁 새로운 프로젝트 구조
```
DataPreprocess_250826/
├── src/                    # 소스코드
├── outputs/               # 출력 폴더
│   └── run_YYYYMMDD_HHMMSS/
│       ├── channels/      # 채널별 분리 데이터
│       ├── plots/         # 시각화 파일
│       ├── processed/     # 전처리된 원본 데이터
│       └── logs/          # 로그 파일
└── Rawdata/               # 원시 데이터
```

In [ ]:
# 라이브러리 및 경로 설정
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime

# 프로젝트 루트 경로 설정
project_root = Path().absolute().parent  # src 폴더의 상위 폴더
sys.path.append(str(project_root / 'src'))

print(f"[경로] 프로젝트 루트: {project_root}")
print(f"[경로] 소스 폴더: {project_root / 'src'}")
print(f"[경로] 출력 폴더: {project_root / 'outputs'}")

In [ ]:
# 개선된 배터리 프로세서 임포트
try:
    from improved_battery_processor import BatteryDataProcessor, Config
    print("[OK] 배터리 프로세서 로드 완료")
except ImportError as e:
    print(f"[오류] 모듈 로드 실패: {e}")
    print("소스 파일이 src/ 폴더에 있는지 확인하세요.")

In [ ]:
# 최신 출력 폴더 찾기 함수
def get_latest_output_folder(outputs_dir):
    """가장 최신의 run_ 폴더를 반환"""
    run_folders = list(outputs_dir.glob('run_*'))
    if not run_folders:
        return None
    
    # 시간순 정렬
    run_folders.sort(key=lambda x: x.name, reverse=True)
    return run_folders[0]

def get_channel_files(channels_dir):
    """채널별 CSV 파일 목록 반환"""
    if not channels_dir.exists():
        return {}
    
    channel_files = {}
    for csv_file in channels_dir.glob('*.csv'):
        # 파일명에서 채널 정보 추출 (예: ...Ch30.csv -> Ch30)
        if '_Ch' in csv_file.stem:
            channel = csv_file.stem.split('_Ch')[-1]  # Ch30
            channel_files[f'Ch{channel}'] = csv_file
    
    return channel_files

print("[함수] 유틸리티 함수 정의 완료")

In [ ]:
# 기존 채널별 데이터 확인
outputs_dir = project_root / 'outputs'

if outputs_dir.exists():
    latest_folder = get_latest_output_folder(outputs_dir)
    
    if latest_folder:
        print(f"[발견] 최신 출력 폴더: {latest_folder.name}")
        
        channels_dir = latest_folder / 'channels'
        channel_files = get_channel_files(channels_dir)
        
        if channel_files:
            print(f"[데이터] 채널별 파일 {len(channel_files)}개 발견:")
            for channel, file_path in channel_files.items():
                file_size = file_path.stat().st_size / (1024*1024)  # MB
                print(f"  {channel}: {file_path.name} ({file_size:.1f}MB)")
        else:
            print("[정보] 채널별 데이터 없음 - 새로 생성 필요")
    else:
        print("[정보] 출력 폴더 없음 - 새로 생성 필요")
else:
    print("[정보] outputs 폴더 없음 - 새로 생성 필요")

In [ ]:
# 채널별 데이터 분석
if 'channel_files' in locals() and channel_files:
    print("[분석] 채널별 데이터 분석 시작...")
    
    channel_summaries = {}
    
    for channel, file_path in channel_files.items():
        try:
            print(f"\n[처리] {channel} 데이터 로드 중...")
            
            # 데이터 로드
            df = pd.read_csv(file_path, encoding='utf-8-sig')
            
            # 기본 정보
            summary = {
                'total_points': len(df),
                'date_range': f"{df['Date'].min()} ~ {df['Date'].max()}",
                'voltage_range': f"{df['Voltage_V'].min():.2f}V ~ {df['Voltage_V'].max():.2f}V",
                'current_range': f"{df['Current_A'].min():.3f}A ~ {df['Current_A'].max():.3f}A",
                'capacity_max': f"{df['Capacity_Ah'].max():.3f}Ah" if 'Capacity_Ah' in df.columns else 'N/A',
                'cycles': len(df['TotalCycle'].unique()) if 'TotalCycle' in df.columns else 'N/A'
            }
            
            channel_summaries[channel] = summary
            
            print(f"  데이터 포인트: {summary['total_points']:,}개")
            print(f"  측정 기간: {summary['date_range']}")
            print(f"  전압 범위: {summary['voltage_range']}")
            print(f"  전류 범위: {summary['current_range']}")
            print(f"  최대 용량: {summary['capacity_max']}")
            print(f"  사이클 수: {summary['cycles']}")
            
        except Exception as e:
            print(f"  [오류] {channel} 처리 실패: {e}")
    
    print(f"\n[완료] {len(channel_summaries)}개 채널 분석 완료")
else:
    print("[건너뛰기] 분석할 채널 데이터가 없습니다.")

In [ ]:
# 채널별 시각화
if 'channel_files' in locals() and len(channel_files) > 0:
    print("[시각화] 채널별 비교 그래프 생성 중...")
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('채널별 배터리 성능 비교', fontsize=16)
    
    colors = ['blue', 'red', 'green', 'orange', 'purple']
    
    for i, (channel, file_path) in enumerate(list(channel_files.items())[:3]):  # 최대 3개 채널
        try:
            df = pd.read_csv(file_path, encoding='utf-8-sig')
            color = colors[i % len(colors)]
            
            # 시간 데이터 준비 (처음 1000개 포인트만)
            sample_df = df.head(1000)
            time_hours = sample_df['Time_Sec'] / 3600.0
            
            # 1. 전압 vs 시간
            axes[0, 0].plot(time_hours, sample_df['Voltage_V'], 
                           color=color, alpha=0.7, linewidth=1, label=channel)
            
            # 2. 전류 vs 시간
            axes[0, 1].plot(time_hours, sample_df['Current_A'], 
                           color=color, alpha=0.7, linewidth=1, label=channel)
            
            # 3. 용량 vs 시간
            if 'Capacity_Ah' in sample_df.columns:
                axes[1, 0].plot(time_hours, sample_df['Capacity_Ah'], 
                               color=color, alpha=0.7, linewidth=1, label=channel)
            
            # 4. 전력 vs 시간
            if 'Power_W' in sample_df.columns:
                axes[1, 1].plot(time_hours, sample_df['Power_W'], 
                               color=color, alpha=0.7, linewidth=1, label=channel)
        
        except Exception as e:
            print(f"  [오류] {channel} 시각화 실패: {e}")
    
    # 축 설정
    axes[0, 0].set_title('전압 vs 시간')
    axes[0, 0].set_xlabel('시간 (hours)')
    axes[0, 0].set_ylabel('전압 (V)')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    axes[0, 1].set_title('전류 vs 시간')
    axes[0, 1].set_xlabel('시간 (hours)')
    axes[0, 1].set_ylabel('전류 (A)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    axes[1, 0].set_title('용량 vs 시간')
    axes[1, 0].set_xlabel('시간 (hours)')
    axes[1, 0].set_ylabel('용량 (Ah)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    axes[1, 1].set_title('전력 vs 시간')
    axes[1, 1].set_xlabel('시간 (hours)')
    axes[1, 1].set_ylabel('전력 (W)')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # 그래프 저장
    if 'latest_folder' in locals() and latest_folder:
        plot_path = latest_folder / 'plots' / 'channel_comparison.png'
        plt.savefig(plot_path, dpi=150, bbox_inches='tight')
        print(f"[저장] 비교 그래프 저장: {plot_path}")
    
    plt.show()
    
else:
    print("[건너뛰기] 시각화할 채널 데이터가 없습니다.")

In [ ]:
# 새로운 데이터 처리 (원시 데이터에서)
raw_data_path = project_root / 'Rawdata' / '250207_250307_3_김동진_1689mAh_ATL Q7M Inner 2C 상온수명 1-100cyc'

if raw_data_path.exists():
    print(f"[새로운 처리] 원시 데이터 처리 시작...")
    print(f"[경로] {raw_data_path}")
    
    try:
        # 새로운 처리를 위한 설정
        config = Config(
            MAX_FILES_PER_CHANNEL=500,  # 더 많은 파일 처리
            PARALLEL_PROCESSING=True,
            MAX_WORKERS=2
        )
        
        processor = BatteryDataProcessor(config)
        
        # 데이터 처리 실행
        results = processor.process_paths([raw_data_path])
        
        if results and results[0].channel_data:
            result = results[0]
            print(f"\n[완료] 새로운 처리 완료!")
            print(f"  포맷: {result.data_format}")
            print(f"  채널 수: {len(result.channel_data)}개")
            
            # 채널별 분리를 위해 프로젝트 구조 정리 스크립트 실행
            print("\n[분리] 채널별 데이터 분리 실행...")
            
            # setup_project_structure 모듈 임포트 및 실행
            from setup_project_structure import ProjectStructureManager
            
            manager = ProjectStructureManager(project_root)
            directories = manager.create_directory_structure()
            separated_files = manager.separate_channels_data(directories)
            
            print(f"[분리완료] 새로운 채널별 데이터:")
            for channel, files in separated_files.items():
                print(f"  {channel}: {len(files)}개 파일")
                for file in files:
                    print(f"    - {file}")
        else:
            print("[오류] 새로운 데이터 처리 실패")
            
    except Exception as e:
        print(f"[오류] 새로운 처리 실패: {e}")
        import traceback
        print(traceback.format_exc())
        
else:
    print("[건너뛰기] 원시 데이터를 찾을 수 없습니다.")
    print(f"[확인] 경로: {raw_data_path}")

## 🎯 완료!

### ✅ 새로운 프로젝트 구조 적용 완료

1. **📁 구조화된 관리**:
   - `src/`: 모든 소스코드 통합 관리
   - `outputs/run_*/`: 타임스탬프별 출력 관리
   - `channels/`: 채널별 데이터 분리 저장

2. **🔄 채널별 분리**:
   - Ch30, Ch31 등 채널별 개별 CSV 파일
   - 각 채널의 독립적 분석 가능
   - 데이터 크기별 최적화

3. **📊 시각화 개선**:
   - 채널별 성능 비교 그래프
   - 전압, 전류, 용량, 전력 다차원 분석
   - 저장 위치 자동 관리

### 💡 사용 방법:
1. **개발**: `src/` 폴더에서 코드 수정
2. **실행**: 이 노트북 또는 스크립트 실행
3. **결과**: `outputs/run_*/` 에서 결과 확인
4. **분석**: `channels/` 에서 채널별 개별 분석

### 🚀 다음 단계:
- 채널별 성능 비교 분석
- 시계열 예측 모델링
- 이상 탐지 및 품질 관리
- 자동화된 리포트 생성